In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import joblib
import random
import sys

from mip import Model as ModelMip, xsum, minimize, maximize, \
INTEGER, BINARY, CONTINUOUS, CutType, OptimizationStatus

import data_generator
from model import StrongStandardNet, StrongVariationalNet
from train import TrainDecoupled, TrainCombined

from sklearn.preprocessing import StandardScaler

import torch.multiprocessing as mp

In [2]:
import cvar_minimization_utils as cmu

In [3]:
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda') 

In [4]:
method_name = 'bnn'
method_learning = 'combined'
aleat_bool = 1

N_SAMPLES = 32
BATCH_SIZE_LOADER = 64

EPOCHS = 100

cpu_count = mp.cpu_count()

seed_number = 0

lr = 0.0005

In [5]:
np.random.seed(seed_number)
torch.manual_seed(seed_number)
random.seed(seed_number)

assert (method_name in ['ann','bnn','gp'])

if method_name in ['ann','bnn']:
    assert (method_learning in ['decoupled','combined'])
    assert (aleat_bool in [True, False])
    assert (N_SAMPLES>=1 and N_SAMPLES<9999)
    #assert (M_SAMPLES>=1 and M_SAMPLES<9999)

bnn = False 
if method_name == 'bnn':
    bnn = True   
    K = 1 # Hyperparameter for the training in ELBO loss
    PLV = 1 # Prior in ELBO loss   

model_name = method_name + '_constrained_'
for i in range(2, len(sys.argv)):
    model_name += '_'+sys.argv[i]
model_name += '_'+ str(seed_number)

In [6]:
def gen_intermidiate(num, n_assets, x1, x2, x3):      
        factor = num * 2/(n_assets)
        return x1**factor + x2**factor + x3**factor
    
def gen_data(N, n_assets, nl, seed_number=42):
    np.random.seed(seed_number)
    x1 = np.random.normal(1, 1, size = N).clip(0)
    x2 = np.random.normal(1, 1, size = N).clip(0)
    x3 = np.random.normal(1, 1, size = N).clip(0)
    X = np.vstack((x1, x2, x3)).T

    Y = np.zeros((N, n_assets))
    for i in range(1, n_assets + 1):
        interm = gen_intermidiate(i, n_assets, x1, x2, x3)
        Y[:,i-1] = (np.sin(interm) - np.sin(interm).mean()) \
        + np.sin(interm).std()*(-0.45 + np.random.random())
        
        ns = Y[:,0].shape[0]//2
        
        Y[:,i-1] = Y[:,i-1] + nl*Y[:,i-1]*x1*(np.random.beta(5, 2, size = Y[:,0].shape) - 0.2)

    return X, Y

def gen_cond_dist(N, n_assets, n_samples, nl, seed_number=420):
    np.random.seed(seed_number)
    Y_dist = np.zeros((n_samples, N, n_assets))
    for i in range(0, n_samples):
        Y_dist[i, :, :] = gen_data(N, n_assets, nl, seed_number=np.random.randint(0,999999))[1]
    return Y_dist

In [7]:
class CVaROP():
    def __init__(self, beta, n_assets, min_return, Y_train):
        self.beta = beta
        self.n_assets = n_assets
        self.R = min_return 
        self.uy = Y_train.mean(axis=0)
        
        self.zstar = np.zeros_like(Y_train)
        self.alphastar = np.zeros_like(Y_train[:,0])
        self.cvarstar = np.zeros_like(Y_train[:,0])
                        
        #for i in range(0, Y_train.shape[0]):
        #    self.zstar[i,:], _ , self.alphastar[i], self.cvarstar[i] = \
        #    self.minimize_cvar(np.expand_dims(Y_train, 0)[:,i,:])
      
    def minimize_cvar(self, y):
    
        n_samples = y.shape[0]
        n_assets = y.shape[1]
        
        assert self.n_assets == n_assets
        
        
        m = ModelMip("cvar")
        m.verbose = 0
        z = ([m.add_var(var_type=CONTINUOUS, name=f'z_{i}') for i in range(0, n_assets)])
        u = ([m.add_var(var_type=CONTINUOUS, name=f'u_{i}') for i in range(0, n_samples)])
        alpha = m.add_var(var_type=CONTINUOUS, name='alpha')

        m.objective = minimize(alpha + (1/(1 - self.beta))*(1/n_samples) \
                               *xsum(u[i] for i in range(0, n_samples)))

        for i in range(0, n_assets):
            m += z[i] >= 0

        for i in range(0, n_samples):
            m += u[i] >= 0

        for i in range(0, n_samples):
            m += xsum(z[j]*y[i][j] for j in range(0, n_assets)) + alpha + u[i] >= 0

        for i in range(0, n_assets):
            m += xsum(-z[i]*self.uy[i] for i in range(0, n_assets)) <= -self.R

        m.optimize()
        f_opt = m.objective_value
        argmins = []
        for v in m.vars:
            argmins.append(v.x)
           
        zstar = argmins[:n_assets]
        ustar = argmins[n_assets:n_assets+n_samples]
        alphastar = argmins[n_assets+n_samples]         

        return zstar, ustar, alphastar, f_opt
    
    
    def minimize_cvar_dataset(self, Y_dist):
        zstar = np.zeros_like(Y_dist[0])
        for i in range(0, Y_dist.shape[1]):
            zstar[i,:] , _, _, _ = self.minimize_cvar(Y_dist[:,i,:])       
        return zstar
    
    def minimize_cvar_given_z(self, y, z):
    
        n_samples = y.shape[0]
        n_assets = y.shape[1]
        
        assert self.n_assets == n_assets
        
        
        m = ModelMip("cvar")
        m.verbose = 0

        u = ([m.add_var(var_type=CONTINUOUS, name=f'u_{i}') for i in range(0, n_samples)])
        alpha = m.add_var(var_type=CONTINUOUS, name='alpha')

        m.objective = minimize(alpha + (1/(1 - self.beta))*(1/n_samples) \
                               *xsum(u[i] for i in range(0, n_samples)))

        for i in range(0, n_samples):
            m += u[i] >= 0

        for i in range(0, n_samples):
            m += xsum(z[j]*y[i][j] for j in range(0, n_assets)) + alpha + u[i] >= 0

        m.optimize()
        f_opt = m.objective_value
        argmins = []
        for v in m.vars:
            argmins.append(v.x)
           
        ustar = argmins[:n_samples]
        alphastar = argmins[n_samples]    

        return ustar, alphastar, f_opt
    
    
    def minimize_cvar_given_z_dataset(self, Y_dist, z):
        alphastar = np.zeros_like(Y_dist[0,:,0])
        for i in range(0, Y_dist.shape[1]):
            _, alphastar[i] , _ = self.minimize_cvar_given_z(Y_dist[:,i,:], z[i,:])
        return alphastar
        
    def calc_cvar_dataset(self, Y_dist, zstar_pred, alpha):
        cvar_pred = alpha + (1/(1-beta))*(np.maximum(-((Y_dist*zstar_pred).sum(2)) \
                                                              - alpha, 0)).mean(0)
        return cvar_pred
        
    
    def calc_f_dataset(self, Y_dist_pred, Y_dist):
        zstar_pred = self.minimize_cvar_dataset(self, Y_dist_pred)
        alphastar_pred = minimize_cvar_given_z_dataset(self, Y_dist, zstar_pred)
        cvar_pred = self.calc_cvar_dataset(Y_dist, zstar_pred)
        return cvar_pred
    
    
    def cost_fn(self, Y_dist_pred, Y_dist):
        f = self.calc_f_dataset(Y_dist_pred, Y_dist)
        f_total = torch.mean(f)
        return f_total
        
    #def end_loss(self, Y_dist_pred, Y_dist):
    #    y_pred = y_pred.unsqueeze(0)
    #    f_total = self.cost_fn(y_pred, y)
    #    return f_total
    
    def end_loss_dist(self, Y_dist_pred, Y_dist):
        f_total = self.cost_fn(Y_dist_pred, Y_dist)
        return f_total

In [8]:
class SolverDiff(torch.autograd.Function):
    @staticmethod
    def forward(ctx, Y_dist_pred):
        
        Y_dist_pred = Y_dist_pred.detach().numpy()
        Y_dist = Ys[:,:,:,1].detach().numpy()
                
        zstar_pred = op_solver_dist.minimize_cvar_dataset(Y_dist_pred)
        alphastar_pred = op_solver_dist.minimize_cvar_given_z_dataset(
            np.expand_dims(Y_original, 0), zstar_pred)
        
            
        ctx.Y_true = Y_true
        ctx.Y = Y
        ctx.argmin = zstar_pred
        ctx.argmin_true = zstar_true
        ctx.sign_f = np.sign(f_pred)
        
        

        return torch.tensor(zstar_pred, dtype=torch.float32)#, torch.tensor(reg, dtype=torch.float32)

    # SPO
    @staticmethod
    def backward(ctx, grad_output):  
        
        zstar_true = np.zeros_like(Y_dist[0])
        for i in range(0, Y_dist.shape[1]):
            zstar_true[i,:] , _, _, _ = self.minimize_cvar(Y_dist[:,i,:])
        
        grad_output = grad_output.detach().numpy()
        Y_aux = 2*ctx.Y - ctx.Y_true
        argmin_aux = op.solve_dataset(Y_to_solve=ctx.ps*Y_aux, activations=ctx.activations, is_torch=False)
        grad_input = (-argmin_aux + ctx.argmin_true)*grad_output/ctx.Y_true
        grad_input = torch.tensor(grad_input, dtype=torch.float32)
                        
        return torch.dstack((grad_input, torch.zeros_like(grad_input))), None

In [9]:
N_train = 1000
N_val = 600
N_test = 600

n_assets = 15

n_samples_orig = 200

nl = 0.1

X, Y_original = gen_data(N_train, n_assets, nl, seed_number)
X_val, Y_val_original = gen_data(N_val, n_assets, nl, seed_number + 80)
X_test, Y_test_original = gen_data(N_test, n_assets, nl, seed_number + 160)

Y_dist = gen_cond_dist(N_train, n_assets, n_samples_orig, nl=nl, seed_number=seed_number)
Y_val_dist = gen_cond_dist(N_val, n_assets, n_samples_orig, nl=nl, seed_number=seed_number + 100)
Y_test_dist = gen_cond_dist(N_test, n_assets, n_samples_orig, nl=nl, seed_number=seed_number + 200)

In [10]:
# Output normalization
scaler = StandardScaler()
scaler.fit(Y_original)
tmean = torch.tensor(scaler.mean_).to(dev)
tstd = torch.tensor(scaler.scale_).to(dev)
joblib.dump(scaler, 'scaler_cvar.gz')

# Function to denormalize the data
def inverse_transform(yy):
    return yy*tstd + tmean

Y = scaler.transform(Y_original).copy()
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)
data_train = data_generator.ArtificialDataset(X, Y)
training_loader = torch.utils.data.DataLoader(
    data_train, batch_size=BATCH_SIZE_LOADER,
    shuffle=True, num_workers=cpu_count)
Y_dist = torch.tensor(Y_dist, dtype=torch.float32)

Y_val = scaler.transform(Y_val_original).copy()
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val_original = torch.tensor(Y_val_original, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)
data_valid = data_generator.ArtificialDataset(X_val, Y_val)
validation_loader = torch.utils.data.DataLoader(
    data_valid, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=cpu_count)

X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test_original = torch.tensor(
    Y_test_original, dtype=torch.float32)
data_test = data_generator.ArtificialDataset(
    X_test, Y_test_original)
test_loader = torch.utils.data.DataLoader(
data_test, batch_size=16,
shuffle=False, num_workers=cpu_count)

In [11]:
n_samples = Y.shape[0]
n_assets = Y.shape[1]
beta = 0.80
min_return = 100

In [40]:
method_name = 'ann'
method_learning = 'decoupled'

bnn = False
aleat_bool = False
if method_name == 'bnn':
    bnn = True
    aleat_bool = True
N_SAMPLES = 16
K = 1

In [41]:
if method_name == 'bnn':
    h = StrongVariationalNet(
    n_samples=N_SAMPLES,
    input_size=X.shape[1], 
    output_size=Y.shape[1], 
    plv=PLV, 
    dev=dev).to(dev)

    
#ANN Baseline model
elif method_name == 'ann':
    h = StrongStandardNet(X.shape[1], Y.shape[1]).to(dev)
    K = 0 # There is no K in ANN
    N_SAMPLES = 1
    
opt_h = torch.optim.Adam(h.parameters(), lr=lr)
mse_loss = nn.MSELoss(reduction='none')


opt_cvar_gZ = cmu.CVaROPgZ(1, n_assets, beta, min_return, torch.tensor(Y_original), dev)
opt_cvar = cmu.CVaRQP(N_SAMPLES, n_assets, beta, min_return, torch.tensor(Y_original), opt_cvar_gZ, dev)

# Decoupled learning approach
if method_learning == 'decoupled':
    train_NN = TrainDecoupled(
                    bnn = bnn,
                    model=h,
                    opt=opt_h,
                    loss_data=mse_loss,
                    K=K,
                    aleat_bool=aleat_bool,
                    training_loader=training_loader,
                    validation_loader=validation_loader,
                    dev=dev
                )

# Combined learning approach (end-to-end loss)
elif method_learning == 'combined':
    train_NN = TrainCombined(
                    bnn = bnn,
                    model=h,
                    opt=opt_h,
                    K=K,
                    aleat_bool=aleat_bool,
                    training_loader=training_loader,
                    scaler=scaler,
                    validation_loader=validation_loader,
                    OP=opt_cvar,
                    dev=dev
                )

In [42]:
model_used = train_NN.train(EPOCHS=200)

  0%|▎                                                                   | 1/200 [00:01<05:11,  1.57s/it]

------------------EPOCH 1------------------
DATA LOSS 	 train 0.908 valid 0.89
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.91 valid 0.89


  1%|▋                                                                   | 2/200 [00:03<05:06,  1.55s/it]

------------------EPOCH 2------------------
DATA LOSS 	 train 0.656 valid 0.677
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.66 valid 0.68


  2%|█                                                                   | 3/200 [00:04<05:03,  1.54s/it]

...

------------------EPOCH 198------------------
DATA LOSS 	 train 0.025 valid 0.163
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.03 valid 0.16


100%|█████████████████████████████████████████████████████████████████▋| 199/200 [05:28<00:01,  1.68s/it]

------------------EPOCH 199------------------
DATA LOSS 	 train 0.025 valid 0.165
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.02 valid 0.16


100%|██████████████████████████████████████████████████████████████████| 200/200 [05:30<00:00,  1.65s/it]

------------------EPOCH 200------------------
DATA LOSS 	 train 0.025 valid 0.161
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.02 valid 0.16


In [43]:
M_opt = 1
model_used.update_n_samples(n_samples=M_opt)
Y_pred = model_used.forward_dist(X_test, aleat_bool)
Y_pred_original = inverse_transform(Y_pred)

In [44]:
opt_cvar = cmu.CVaRQP(M_opt, n_assets, beta, min_return, torch.tensor(Y_original), opt_cvar_gZ, dev)
opt_cvar.end_loss_dist(Y_pred_original, Y_test_original)

tensor(27.0925, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [39]:
M_opt = 1
model_used.update_n_samples(n_samples=M_opt)
Y_pred_val = model_used.forward_dist(X_val, aleat_bool)
Y_pred_val_original = inverse_transform(Y_pred_val)

opt_cvar = cmu.CVaRQP(M_opt, n_assets, beta, min_return, torch.tensor(Y_original), opt_cvar_gZ, dev)
opt_cvar.end_loss_dist(Y_pred_val_original, Y_val_original)

tensor(8.4273, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [26]:
opt_cvar.end_loss_dist(Y_pred_original, Y_test_original)

tensor(42.9886, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [49]:
opt_cvar.end_loss_dist(Y_pred_original, Y_test_original)

tensor(55.0741, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [18]:
opt_cvar = cmu.CVaRQP(M_opt, n_assets, beta, min_return, torch.tensor(Y_original), opt_cvar_gZ, dev)
opt_cvar.end_loss_dist(Y_pred_original, Y_test_original)

tensor(63.7008, dtype=torch.float64, grad_fn=<MeanBackward0>)

tensor(41.8511, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [27]:
Y_pred_original.shape

torch.Size([1, 1000, 20])

In [50]:
opt_cvar = cmu.CVaRQP(M_opt, n_assets, beta, min_return, torch.tensor(Y_original), opt_cvar_gZ, dev)
zz = opt_cvar.forward(torch.permute(Y_pred_original, (1, 0, 2)))

In [56]:
Y_test_original.shape

torch.Size([600, 10])

In [66]:
(Y_test_original*zz).sum(1).clip(None, 0).mean()

tensor(-67.2309, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [53]:
zz[0]

tensor([1.6260e+01, 4.9736e+01, 2.2901e-14, 8.6343e+01, 2.8042e+01, 6.7583e+01,
        5.7734e+01, 5.1143e+00, 1.2606e+01, 6.7443e-14], dtype=torch.float64,
       grad_fn=<SelectBackward0>)

In [40]:
Y_pred_original.shape

torch.Size([32, 600, 10])

In [42]:
opt_cvar = cmu.CVaRQP(1, n_assets, beta, min_return, torch.tensor(Y_original), opt_cvar_gZ, dev)
opt_cvar.end_loss_dist(Y_test_original.unsqueeze(0), Y_test_original)

tensor(10.6740, dtype=torch.float64)

In [33]:
Y_original.mean(), Y_original.std(), Y_original.max(), Y_original.min()

(0.13693512093163707,
 0.8841071259180603,
 2.6387125058604113,
 -2.7960391110496476)

In [25]:
Y_pred_original.shape

torch.Size([32, 600, 10])

In [22]:
opt_cvar.end_loss_dist(Y_pred_original, torch.permute(torch.tensor(Y_test_dist), (1, 0, 2)))

RuntimeError: The size of tensor a (3) must match the size of tensor b (202) at non-singleton dimension 1

In [22]:
zstar_pred_new = opt_cvar.forward(torch.permute(Y_pred_original, (1,0,2)))

In [23]:
zstar_pred_new

tensor([[ 7.3330e+01,  8.4389e+01,  1.1276e+02,  2.1225e-14,  1.1709e+02],
        [ 6.7217e+01,  9.0927e+01,  1.1521e+02,  2.0232e-14,  1.1385e+02],
        [ 1.1074e+01,  1.0179e+02, -5.4336e-14, -2.2984e-14,  2.5212e+02],
        [ 1.5634e+01,  1.3213e+02,  8.3355e+01,  4.7789e+01,  1.6134e+02]],
       dtype=torch.float64, grad_fn=<SliceBackward0>)

In [24]:
alphastar_pred_new = opt_cvar.OP_Z.forward(torch.permute(Y_test_original.unsqueeze(0), (1,0,2)), zstar_pred_new)

In [25]:
alphastar_pred_new

tensor([1.6425e+02, 9.3361e+01, 1.0529e-14, 4.5766e-15], dtype=torch.float64,
       grad_fn=<SelectBackward0>)

In [36]:
zstar_pred = op_old.minimize_cvar_dataset(Y_pred_original.detach().numpy())
alphastar_pred = op_old.minimize_cvar_given_z_dataset(np.expand_dims(Y_test_original, 0), zstar_pred)
cvar_pred = op_old.calc_cvar_dataset(np.expand_dims(Y_test_original, 0), zstar_pred, alphastar_pred)

> /tmp/ipykernel_256377/2293865252.py(59)minimize_cvar()
     57 
     58 
---> 59         return zstar, ustar, alphastar, f_opt
     60 
     61 

ipdb> l
     54 
     55         import pdb
     56         pdb.set_trace()
     57 
     58 
---> 59         return zstar, ustar, alphastar, f_opt
     60 
     61 
     62     def minimize_cvar_dataset(self, Y_dist):
     63         zstar = np.zeros_like(Y_dist[0])
     64         for i in range(0, Y_dist.shape[1]):

ipdb> zstar
[0.0, 461.23839069272395, 0.0, 0.0, 0.0]
ipdb> ustar
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ipdb> alphastar
0.0
ipdb> f_opt
0.0
ipdb> c
> /tmp/ipykernel_256377/2293865252.py(59)minimize_cvar()
     57 
     58 
---> 59         return zstar, ustar, alphastar, f_opt
     60 
     61 

ipdb> zstar
[0.0, 0.0, 345.19590618039325, 0.0, 0.0]
ipdb> f_opt
0.0
ipdb> c
> /tmp/ipykernel_256377/2293865252.py(59)minimize_cvar()
     57 
     58 
---> 59         return zstar, ustar, alphastar, f_opt
     60 
     61 

ipdb> f_

BdbQuit: 

In [32]:
zstar_pred

array([[  0.        , 461.23839069,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        , 345.19590618,   0.        ,
          0.        ],
       [ 11.07380712, 101.79467917,   0.        ,   0.        ,
        252.11989681],
       [ 15.63351324, 132.13076345,  83.35474558,  47.78917676,
        161.33640877]])

In [28]:
alphastar_pred

array([346.91013, 280.73404,   0.     ,   0.     ], dtype=float32)

In [30]:
cvar_pred.mean()

156.91104125976562

In [37]:
op_old.uy

array([ 0.18839605,  0.21680762,  0.28969057, -0.08941982,  0.30082463])

In [24]:
(zstar_pred*op_old.uy).sum(axis=1)

array([100., 100., 100., 100.])

In [25]:
(zstar_pred_new*opt_cvar.uy).sum(axis=1)

tensor([101.0000, 101.0000, 100.0000, 100.0000], dtype=torch.float64,
       grad_fn=<SumBackward1>)

In [35]:
((Y_pred_original.mean(0)).detach().numpy()*zstar_pred).sum(1)

array([384.86705589, 214.75938277,  69.39420483,  -6.3605234 ])

In [26]:
zstar_pred

array([[  0.        , 461.23839069,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        , 345.19590618,   0.        ,
          0.        ],
       [ 11.07380712, 101.79467917,   0.        ,   0.        ,
        252.11989681],
       [ 15.63351324, 132.13076345,  83.35474558,  47.78917676,
        161.33640877]])

In [ ]:
zstar_pred = opt_cvar.forward(Y_pred_original)
alphastar_pred = op_old.minimize_cvar_given_z_dataset(np.expand_dims(Y_test_original, 0), zstar_pred)
cvar_pred = op_old.calc_cvar_dataset(np.expand_dims(Y_test_original, 0), zstar_pred, alphastar_pred)

In [47]:
(zstar_pred_new[1].detach().numpy() - zstar_pred[1])

array([6.16415948e-15, 3.94899537e+03, 3.62644610e+03, 4.84417479e+03,
       4.83752559e+03, 4.89291048e+03, 8.26435561e+03, 5.53094399e+03,
       5.34771620e+03, 3.18915729e+03, 4.30792665e+03, 5.98957556e+03,
       4.85176412e+03, 2.29808733e-12, 2.52292449e+03, 4.08726661e+03,
       1.79131885e+03, 2.77712095e-12, 3.26970287e-13, 6.33605259e-13])

In [58]:
zstar_pred_new[5]

tensor([1.0777e+03, 4.3437e+03, 2.3827e+03, 4.3540e+03, 5.5204e+03, 4.4254e+03,
        5.3152e+03, 7.9586e+03, 5.0483e+03, 5.0448e+03, 4.2246e+03, 4.5593e+03,
        4.4394e+02, 1.9223e+03, 2.8281e-12, 3.8541e+02, 1.2015e+03, 1.3990e-12,
        8.0155e+02, 4.0557e+03], dtype=torch.float64,
       grad_fn=<SelectBackward0>)

In [59]:
zstar_pred[5]

array([  0.        , 499.72849577,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

In [30]:
cvar_pred

array([199.69296265,   0.        ,   0.        , ...,   0.        ,
         0.        ,   0.        ])

In [ ]:
cvar_pred.mean()

In [37]:
zstar_test = op_solver_dist.minimize_cvar_dataset(np.expand_dims(Y_test_original, 0))
alphastar_test = op_solver_dist.minimize_cvar_given_z_dataset(np.expand_dims(Y_test_original, 0), zstar_test)
cvar_test = op_solver_dist.calc_cvar_dataset(np.expand_dims(Y_test_original, 0), zstar_test, alphastar_test)

In [38]:
cvar_test.mean()

2.130949

In [81]:
cvar_pred = alphastar_pred + (1/(1-beta))*(np.maximum(-((Y_dist*zstar_pred).sum(2)) - alphastar_pred, 0)).mean(0)

In [82]:
f_pred_dataset = cvar_pred.mean()

In [83]:
fair_regret = f_pred_dataset - op_solver.cvarstar.mean()

In [84]:
fair_regret

4.802389564712255

In [15]:
op_solver.alphastar + (1/(1-beta))*(np.maximum(-((Y_dist*op_solver.zstar).sum(2)) - op_solver.alphastar, 0)).mean(0)

array([14.53573779, 15.40195475, 16.40051164, 15.38380813, 14.62208475,
       17.06472651, 16.73093918, 15.48154203, 15.9300598 , 16.2041831 ,
       15.73938929, 16.89708193, 15.03783049, 15.81537838, 16.31737609,
       15.29510803, 16.09471929, 14.65658093, 15.2551622 , 15.96507641,
       17.64164752, 17.32653127, 14.25618879, 13.61590662, 16.92963039,
       15.39177444, 15.94804965, 15.49853349, 17.32618356, 15.23503582,
       15.87627014, 15.36836605, 16.97589351, 14.78739717, 16.26122729,
       18.07661054, 15.84807938, 14.6334357 , 16.64567533, 15.43718012,
       15.94837282, 14.39151073, 16.18164058, 14.63956316, 16.72973545,
       17.41537503, 15.88181541, 16.850747  , 15.30816212, 13.10728099,
       13.95678749, 15.29048891, 15.62895434, 16.26142701, 15.8916077 ,
       15.7978594 , 16.34415953, 15.10308489, 15.14827456, 15.24617177,
       16.37071953, 15.95829432, 14.10051356, 14.39860641, 14.34294722,
       15.74917112, 14.66477814, 15.33982245, 16.17635938, 14.59

In [ ]:
def compute_cvar(alpha, z, y):
    

In [154]:
1/(Y_dist.shape[0]*(1-beta)) * Y_dist

(1000, 1000, 20)

In [ ]:
Y_dist

In [159]:
Y_dist*zstar

array([[[-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-0.00000000e+00, -8.01762101e-02,  0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00,  7.26688064e-01],
        [-2.77114335e-01, -3.74680090e-01, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -3.38799630e+00],
        ...,
        [-1.75644780e-02, -2.58917253e+00, -0.00000000e+00, ...,
          5.05400752e+00, -3.07404112e-01, -3.07341864e+00],
        [-5.71981859e-01, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00,  1.60096702e-01,  0.00000000e+00],
        [-3.02333994e-02, -0.00000000e+00, -0.00000000e+00, ...,
          0.00000000e+00,  2.50841331e+00,  7.44609135e-02]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.62525526e-01,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e

In [155]:
Y_dist.shape[0]

1000

In [151]:
Y_pred.shape

(50, 1000, 20)

In [137]:
Y_pred

array([[[-0.17638682, -0.09089706, -0.3830332 , ...,  0.06720436,
          0.42834576,  0.26608252],
        [ 0.32939309,  0.55933907,  0.0138017 , ..., -0.38775832,
          0.38721256,  0.48453956],
        [-0.08600242, -0.06805484,  0.51201416, ..., -0.30176222,
         -0.30011163,  0.48041784],
        ...,
        [ 0.5547552 ,  0.53806358,  0.41646602, ..., -0.1646881 ,
         -0.38714016,  0.0585669 ],
        [-0.259434  , -0.03745498, -0.23963089, ...,  0.41290765,
          0.06538298, -0.25733343],
        [ 0.1752564 ,  0.55979906, -0.1785693 , ...,  0.22204114,
          0.43828789, -0.28569533]],

       [[ 0.25746872, -0.39684173, -0.3284593 , ...,  0.3906845 ,
         -0.39183591,  0.16925576],
        [-0.01981441, -0.05351203,  0.32120704, ...,  0.0888863 ,
          0.13828446,  0.25011441],
        [-0.23420105, -0.14614686,  0.52928029, ...,  0.4170639 ,
          0.55037046,  0.51355313],
        ...,
        [ 0.54040521,  0.23890133, -0.31902953, ..., -

In [8]:
Y_train[:50].mean(0)

array([ 0.01103508,  0.18582536,  0.02757531,  0.176867  ,  0.28287247,
        0.24011691,  0.10373269,  0.22923871,  0.30873738, -0.07671305,
        0.39660958, -0.21470825,  0.23993704, -0.19205835,  0.06976262,
       -0.07803554,  0.40508948, -0.03724071,  0.02091073,  0.40063581])

In [9]:
Y_train[:50].std(0)

array([0.41032437, 0.46137744, 0.52203211, 0.58222118, 0.63199747,
       0.66359498, 0.67581864, 0.67831638, 0.68665954, 0.70163281,
       0.70021433, 0.67717812, 0.68244412, 0.71339072, 0.69029142,
       0.6841247 , 0.67276091, 0.62925573, 0.6156747 , 0.7166073 ])

In [10]:
op_solver.minimize_cvar(Y_train[:50])

([0.0,
  0.0,
  0.0,
  0.0,
  10.93680879473024,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  5.475523075674423,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  4.652755334066428,
  0.0,
  0.0,
  3.8449485359989164],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.27095111058335786,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 3.958868157503432,
 4.067248601736775)

In [34]:
y = Y_train

In [133]:
y.mean(axis=0)

array([ 0.09154903,  0.12018547, -0.00612775,  0.08615439,  0.07785487])

In [125]:
y.std(axis=0)

array([0.28204021, 0.29966942, 0.2348671 , 0.25421325, 0.31059858])

In [126]:
m.status.name

'OPTIMAL'

In [127]:
alpha_opt = -1

In [128]:
(1/n_samples)*(1/(1-beta))*np.sum(np.clip(((-y*np.array(argmins[:5])).sum(axis=1) - alpha_opt), 0, None))

71.47976078082966

In [129]:
(y.mean(axis=0)*np.array(argmins[:5])).sum()

9.999999999999998

In [134]:
argmins

[53.23569452514242,
 18.33912369004899,
 0.0,
 30.08292387425699,
 4.372902220899407,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.4596077782392385,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 17.568190336620138]

In [135]:
f_opt

18.152033447915834

In [3]:
(1/(1 - B))*(1/n)

1.9999999999999982

In [ ]:


class LPKnapsackProblem():
    def __init__(self, Y, probs_mean):
        self.Y = Y
        self.probs_mean = probs_mean
        self.n_items = self.Y.shape[1]
        self.weights, self.capacity = self.define_knapsack()
        
    def define_knapsack(self):
        weights = self.Y.mean(axis=0)*self.probs_mean + 80*np.random.random(size=self.n_items)
        capacity = np.random.uniform(0.4, 0.6) * sum(weights)
        return weights, int(capacity)
    
    def solve_knapsack_mip(self, values, activations):
        m = ModelMip("knapsack")
        m.verbose = 0
        z = ([m.add_var(var_type=CONTINUOUS) for i in range(0, self.n_items)])
        m.objective = minimize(
            -xsum(values[i] * z[i] for i in range(0, self.n_items)))
        
        for i in range(0, self.n_items):
            m += z[i] <= 1
            m += -z[i] <= 0
            
        m += xsum(
            self.weights[i] * z[i] for i in range(0, self.n_items)) <= self.capacity
        m += xsum(z[i] for i in range(0, self.n_items) if activations[i]==0) <= 0
        
        m.optimize()
        f_opt = m.objective_value
        argmins = []
        for v in m.vars:
            argmins.append(v.x)
        return argmins, f_opt